## Import Tensorflow

In [1]:
import tensorflow as tf

### Tensors

A **Tensor** is a multi-dimensional array. Similar to Numpy `ndarray`.

In [2]:
print(tf.add(1, 2))
print(tf.add([1, 2], [3, 4]))
print(tf.square(5))
print(tf.reduce_sum([1, 2, 3]))

tf.Tensor(3, shape=(), dtype=int32)
tf.Tensor([4 6], shape=(2,), dtype=int32)
tf.Tensor(25, shape=(), dtype=int32)
tf.Tensor(6, shape=(), dtype=int32)


In [3]:
print(tf.square(2) + tf.square(13))

tf.Tensor(173, shape=(), dtype=int32)


In [4]:
x = tf.matmul([[1]], [[2, 3]])
print(x)
print(x.shape)
print(x.dtype)

tf.Tensor([[2 3]], shape=(1, 2), dtype=int32)
(1, 2)
<dtype: 'int32'>


#### Numpy Compatibility

Converting between a Tensorflow `tf.Tensor`s and a Numpy `ndarray` is easy:
- Tensorflow operations automatically convert Numpy ndarrays to Tensors.
- Numpy operations automatically convert Tensors to Numpy ndarrays.

In [5]:
import numpy as np

ndarray = np.ones([3, 3])

print("TensorFlow operations convert numpy arrays to Tensors automatically")
tensor = tf.multiply(ndarray, 42)
print(tensor)

print("And NumPy operations convert Tensors to numpy arrays automatically")
print(np.add(tensor, 1))

print("The .numpy() method explicitly converts a Tensor to a numpy array")
print(tensor.numpy())

TensorFlow operations convert numpy arrays to Tensors automatically
tf.Tensor(
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]], shape=(3, 3), dtype=float64)
And NumPy operations convert Tensors to numpy arrays automatically
[[43. 43. 43.]
 [43. 43. 43.]
 [43. 43. 43.]]
The .numpy() method explicitly converts a Tensor to a numpy array
[[42. 42. 42.]
 [42. 42. 42.]
 [42. 42. 42.]]


## GPU acceleration

In [6]:
x = tf.random.uniform([3, 3])

print("Is there a GPU available: ")
print(tf.config.experimental.list_physical_devices("GPU"))

print("Is the Tensor on GPU #0: ")
print(x.device.endswith('GPU:0'))

Is there a GPU available: 
[]
Is the Tensor on GPU #0: 
False


### Explicit Device Placement

In [7]:
import time

def time_matmul(x):
    start = time.time()
    for loop in range(10):
        tf.matmul(x, x)
    result = time.time() - start
    
    print(f"10 loops: {1000*result:0.2f}ms")
    
print("On CPU: ")
with tf.device("CPU:0"):
    x = tf.random.uniform([1000, 1000])
    assert x.device.endswith("CPU:0")
    time_matmul(x)
    
if tf.config.experimental.list_physical_devices("GPU"):
    print("On GPU: ")
    with tf.device("GPU:0"):
        x = tf.random.uniform([1000, 1000])
        assert x.device.endswith("GPU:0")
        time_matmul(x)

On CPU: 
10 loops: 593.75ms


## Datasets

In [8]:
ds_tensors = tf.data.Dataset.from_tensor_slices([1, 2, 3, 4, 5, 6])

import tempfile
_, filename = tempfile.mkstemp()

with open(filename, 'w') as f:
    f.write("""Line 1
    Line 2
    Line 3
    """)
    
ds_file = tf.data.TextLineDataset(filename)
ds_file

<TextLineDatasetV2 shapes: (), types: tf.string>

In [9]:
ds_tensors = ds_tensors.map(tf.square).shuffle(2).batch(2)

ds_file = ds_file.batch(2)

In [10]:
print("Elements of ds_tensors:")
for x in ds_tensors:
    print(x)
    
print("\nElements in ds_file:")
for x in ds_file:
    print(x)

Elements of ds_tensors:
tf.Tensor([4 9], shape=(2,), dtype=int32)
tf.Tensor([16 25], shape=(2,), dtype=int32)
tf.Tensor([ 1 36], shape=(2,), dtype=int32)

Elements in ds_file:
tf.Tensor([b'Line 1' b'    Line 2'], shape=(2,), dtype=string)
tf.Tensor([b'    Line 3' b'    '], shape=(2,), dtype=string)
